In [ ]:
# MCQ: MedQA (5% = 510)
# https://huggingface.co/datasets/medalpaca/medical_meadow_medqa/

In [ ]:
# Context Retrieval (OpenAI)

In [1]:
## Env setup
import getpass
import os
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

# LLM
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_BASE_URL"] = os.getenv('OPENAI_BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini")

# Neo4j env (lmkg)
url = "neo4j://43.140.200.9:7687"
username ="neo4j"
password = "20230408"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    refresh_schema=False
)

# CSV env
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from dotenv import load_dotenv
import pandas as pd

sys.path.append(r'C:\Users\Sin Yee\Desktop\rag_techniques')

C:\Users\Sin Yee\AppData\Local\Temp\ipykernel_29624\3432849121.py:21: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [ ]:
## Query & Knowledge Agent Graph
## Helper utilities
from typing import List, Optional, Literal, TypedDict, Union
from langchain_core.language_models.chat_models import BaseChatModel
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages

# Query agent
class QState(MessagesState):
    next: str
    planner_executed: bool = False

# Knowledge agent
class State(MessagesState):
    next: str
    original_question: str = ""
    references: dict = {}
    completed_agents: list = []
    excluded_agents: list = []

def make_supervisor_node(llm: BaseChatModel, members: list[str], agent_scopes: dict = None) -> callable:
    """
    Create supervisor node that only routes between agents for reference retrieval
    """
    if agent_scopes is None:
        agent_scopes = {
            "lmkg": "General medical queries using knowledge graph: diseases, exams, indicators, symptoms, complications etc.",
            "hkg": "For questions on symptom.",
            "ds": "For questions on symptoms or treatments.",
            "primekg": "To retrieve drugs indicated for diseases.",
            "drugreviews": "To retrieve drugs along with patient reviews.",
            "wiki": "For disease definitions and overviews.",
            "mayoclinic": "For clinical info: causes, treatments, symptoms, complications.",
            "llmself": "Fallback only if others are irrelevant."
        }
    
    def supervisor_node(state: State) -> Command:
        """Routes to agents for reference retrieval only."""
        # Get the original question
        if not state.get("original_question"):
            original_question = next(
                msg.content for msg in state["messages"] 
                if msg.type == "human"
            )
        else:
            original_question = state["original_question"]
        
        completed_agents = state.get("completed_agents", [])
        references = state.get("references", {})
        excluded_agents = state.get("excluded_agents", [])

        # Fallback REPLACEMENT logic
        if "llmself" not in completed_agents and "llmself" in members:
            for agent in completed_agents:
                ref = references.get(agent, "")
                # Check for both: No information or empty cypher retrieved_result
                is_no_info = ref == "No information retrieved"
                is_empty_cypher = (
                    isinstance(ref, dict)
                    and "generated_cypher" in ref  # identify as cypher_reference
                    and (
                        not ref.get("retrieved_result")
                        or (isinstance(ref.get("retrieved_result"), list) and not any(str(r).strip() for r in ref["retrieved_result"]))
                    )
                )
                if is_no_info or is_empty_cypher:
                    print(f"⚠️ Replacing low-quality agent '{agent}' with fallback agent 'llmself'.")
                    
                    # Remove low-quality agent
                    completed_agents.remove(agent)
                    references.pop(agent, None)
                    excluded_agents.append(agent)  # Mark as excluded
                    
                    return Command(goto="llmself", update={
                        "next": "llmself",
                        "original_question": original_question,
                        "completed_agents": completed_agents,
                        "excluded_agents": excluded_agents
                    })

        # Check if we have references from 2 agents - if so, finish.
        if len(completed_agents) >= 2:
            return Command(goto=END, update={
                "next": "FINISH",
                "original_question": original_question
            })
        
        available_agents = [agent for agent in members]
        remaining_agents = [agent for agent in available_agents 
                            if agent not in completed_agents and agent not in excluded_agents]
        
        # Check if all agents have been completed
        if not remaining_agents:
            return Command(goto=END, update={
                "next": "FINISH",
                "original_question": original_question
            })
        
        # Create agent scope descriptions for remaining agents
        remaining_agent_scopes = {agent: agent_scopes.get(agent, "General purpose agent") 
                                for agent in remaining_agents}
        
        # Create options with only remaining agents
        options = ["FINISH"] + remaining_agents
        
        # System prompt for agent selection
        system_prompt_updated = (
            f"You are a supervisor selecting the top 2 agents for medical reference retrieval. "
            f"Choose one agent at a time based on query relevance.\n\n"
            f"Agent options and specialties:\n"
        )
        
        for agent, scope in remaining_agent_scopes.items():
            system_prompt_updated += f"- {agent}: {scope}\n"
        
        system_prompt_updated += (
            f"\nCompleted agents: {completed_agents} (Target: 2 agents)\n"
            f"Available agents: {remaining_agents}\n\n"
            f"GOAL: Select the 2 MOST RELEVANT agents.\n"
            f"Progress: {len(completed_agents)}/2 done\n\n"
            f"Instructions:\n"
            f"1. If 2 agents are done, select FINISH\n"
            f"2. Choose the most relevant agent from remaining\n"
            f"3. Prioritize by question type:\n"
            f"   - Symptoms: hkg > ds > mayoclinic\n"
            f"   - Causes: mayoclinic > wiki\n"
            f"   - Diagnosis: mayoclinic > ds\n"
            f"   - Medications: drugreviews > primekg > lmkg\n"
            f"   - Treatments: ds > mayoclinic \n"
            f"   - Complications: lmkg > mayoclinic\n"
            f"   - Examinations/tests: lmkg > ds\n"
            f"   - Definition/overview: wiki > mayoclinic\n"
            f"   - Fallback (only if others unsuitable): llmself\n"
            f"4. Pick the next best from: {remaining_agents}\n\n"
            f"DO NOT select any agents not listed in 'Valid responses' below.\n\n"
            f"Valid responses: {options}\n"
            f"Use 2 best agents, then FINISH."
        )

        messages = [
            {"role": "system", "content": system_prompt_updated},
            {"role": "user", "content": f"Question: {original_question}\n\nSelect the most relevant agent:"}
        ]
        
        class Router(TypedDict):
            """Worker to route to next. Select the most relevant agent for reference retrieval."""
            next: str
            
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        
        print(f"Question: {original_question}")
        print(f"Remaining agents: {remaining_agents}")
        print(f"LLM selected: {goto}")
        print(f"Valid options: {options}")
        
        # Validate that the response is in our options
        if goto not in options:
            print(f"Warning: Invalid selection '{goto}'. Defaulting to FINISH.")
            goto = "FINISH"
            
        if goto == "FINISH":
            goto = END
            
        return Command(goto=goto, update={
            "next": goto, 
            "original_question": original_question
        })
    
    return supervisor_node

## Tools (remain the same for retrieval)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate 
from langchain_core.prompts import ChatPromptTemplate

def planner_tool():
    planner_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a query planning agent for a medical knowledge system. 
        Determine if the user's question is:
        
        1. "single-step" - Direct factual questions
           e.g., "What is diabetes?", "What are symptoms of flu?", "Which drug treats malaria?"
        
        2. "multi-step" - Complex questions needing multiple facts or reasoning
           e.g., "Compare treatment options for diabetes vs hypertension", 
           "What's the best treatment for hypertension in a diabetic patient with renal impairment?"
        
        Respond with either "single-step" or "multi-step"."""),
        ("human", "{input}")
    ])
    planner_chain = LLMChain(llm=llm, prompt=planner_prompt)

    def planner_agent(question: str):
        return planner_chain.invoke({"input": question})['text']
    return planner_agent

## Tools for reference retrieval only
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA # vector
from langchain.chains import GraphCypherQAChain # cypher
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate 

def lmkg_retrieval_tool():
    # A) Tool's helper function
    def build_schema(graph):
        node_info = graph.query("""
            MATCH (n)
            WITH labels(n) AS label_list, keys(n) AS props
            UNWIND label_list AS label
            RETURN DISTINCT label, collect(DISTINCT props) AS property_keys
        """)
        
        rel_info = graph.query("""
            MATCH ()-[r]->()
            RETURN DISTINCT type(r) AS rel_type, keys(r) AS property_keys
        """)

        # Build string
        schema = "Node properties are the following:\n"
        for row in node_info:
            label = row['label']
            props = ', '.join(row['property_keys'][0]) if row['property_keys'] else ''
            schema += f"{label} {{{props}}}\n"

        schema += "\nThe relationships are the following:\n"
        for row in rel_info:
            rel_type = row['rel_type']
            schema += f"(:X)-[:{rel_type}]->(:Y)\n"

        return schema

    # B) Vector search
    lmkg_vector = FAISS.load_local(
        "lmkg_faiss",
        OpenAIEmbeddings(model='text-embedding-3-small'),
        allow_dangerous_deserialization=True)

    from typing import Union, Dict, Any
    
    def vector_search(input_data: Union[str, Dict[str, Any]]) -> dict:
        """Returns retrieved documents and final answer."""
        question = input_data if isinstance(input_data, str) else input_data.get("input", "")
        if not question:
            return {"error": "No question provided"}

        # Retrieve documents
        retrieved_docs = lmkg_vector.similarity_search(question, k=4)
        retrieved_context = [doc.page_content for doc in retrieved_docs]

        return {
            "query": question,
            "retrieved_context": retrieved_context
        }


    # C) Cypher search
    CYPHER_GENERATION_TEMPLATE = """
    Task: Generate precise Cypher query to answer the question:
    {question}

    Requirements:
    1. MUST start from node ID: {node_id} using WHERE id(n) = {node_id}
    2. MUST wrap labels/relationships with spaces in backticks.  
    E.g., [:`Active Ingredient`], [:Complication]
    3. Use only ONE relationship type
    4. Return ONLY what's needed to answer the question
    5. Use this schema:

    Schema:
    {schema}

    Return ONLY the executable Cypher query with no additional text.
    """

    CYPHER_GENERATION_PROMPT = PromptTemplate(
        input_variables=["schema", "question", "node_id"], template=CYPHER_GENERATION_TEMPLATE
    )

    cypher_chain = GraphCypherQAChain.from_llm(
        cypher_llm = llm,
        qa_llm = llm, graph=graph, verbose=True,
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        return_direct=True, # bypass qa_llm
        return_intermediate_steps=True, # return cypher query & retrieved context
        allow_dangerous_requests=True
    )

    # insert graph_schema
    cypher_chain.graph_schema = build_schema(graph)
    schema = build_schema(graph)

    # D) LMKG Agent
    # Use vector search & cypher query to explore KG
    from langchain.agents import Tool, AgentExecutor, create_structured_chat_agent
    from langchain import hub
    from typing import Dict, Any, Union

    # Wrap cypher chain in a function with proper input handling
    def cypher_search(input_data: Union[str, Dict[str, Any]]) -> dict:
        """Executes Cypher and returns query, raw result, and LLM answer."""
        try:
            question = input_data if isinstance(input_data, str) else input_data.get("input", "")
            if not question:
                return {"error": "No question provided"}

            similar_nodes = lmkg_vector.similarity_search(question, k=1)
            if not similar_nodes:
                return {"error": "No matching nodes found"}

            node_id = similar_nodes[0].metadata['node_id']
            print(f"Retrieved node id: {node_id}")

            # Inject into GraphCypherQAChain
            cypher_response = cypher_chain({
                "query": question,
                "schema": schema,
                "node_id": node_id
            })

            print("cypher_response:")
            print(cypher_response)

            intermediate = cypher_response.get("intermediate_steps", [])
            
            generated_cypher = next((step["query"] for step in intermediate if "query" in step), None)
            retrieved_context = cypher_response.get("result", [])

            return {
                "query": question,
                "node_id": node_id,
                "generated_cypher": generated_cypher,
                "retrieved_result": retrieved_context
            }

        except Exception as e:
            return {"error": str(e)}

    tools = [
        Tool(
            name="med_vector_qa",
            func=vector_search,
            description="""Use for open-ended, semantic, or fuzzy medical questions requiring
            contextual understanding from vector-based retrieval.
            Examples:
            "What is ADHD?"
            "Explain the mechanism of diabetes."
            "Long-term effects of COVID-19?"
            Only use this tool once.
            """,
        ),
        Tool(
            name="med_cypher_qa",
            func=cypher_search,
            description=""""Use for specific, factual medical queries (about diseases, symptoms, complications, 
            affected organs, treatments, drugs etc.) using structured graph data. 
            Examples:
            "What is the symptom of ADHF?"
            "Organs affected by HIV?"
            "Complications of diabetes?"
            "Drug for tuberculosis?"
            Only use this tool once.
            """,
        ),
    ]

    prompt = hub.pull("hwchase17/structured-chat-agent")
    agent = create_structured_chat_agent(llm, tools, prompt)

    # Create agent executor
    lmkg_agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)
    return lmkg_agent_executor, lmkg_vector

def hkg_retrieval_tool():
    """HKG tool for reference retrieval only"""
    hkg_vector = FAISS.load_local(
        "faiss_hkg",
        OpenAIEmbeddings(model='text-embedding-ada-002'),
        allow_dangerous_deserialization=True)
    return hkg_vector

def ds_retrieval_tool():
    ds_vector = FAISS.load_local(
        "diseases_symptoms_faiss",
        OpenAIEmbeddings(model='text-embedding-ada-002'),
        allow_dangerous_deserialization=True)
    return ds_vector

def primekg_retrieval_tool():
    """PrimeKG tool for reference retrieval only"""
    primekg_vector = FAISS.load_local(
        "primekg_faiss",
        OpenAIEmbeddings(model='text-embedding-ada-002'),
        allow_dangerous_deserialization=True)
    return primekg_vector

def drugreviews_retrieval_tool():
    drugreviews_vector = FAISS.load_local(
        "drug_reviews_faiss",
        OpenAIEmbeddings(model='text-embedding-ada-002'),
        allow_dangerous_deserialization=True)
    return drugreviews_vector

def wiki_retrieval_tool():
    """Wiki tool retrieval"""
    from openai import OpenAI

    client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url=os.environ["DEEPSEEK_BASE_URL"])

    def extract_entity_from_query(query):
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "user", "content": f"What is the main entity or concept in this query: '{query}'? Extract only the core term (e.g., 'hyponatremia', not 'causes of hyponatremia'). Return only the entity name in one line."}]
        )
        entity = response.choices[0].message.content.strip()
        print(f'Extracted entity: {entity}')
        return entity.replace(" ", "_")
    
    return extract_entity_from_query

def mayoclinic_retrieval_tool():
    """MayoClinic tool retrieval"""
    from openai import OpenAI

    client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url=os.environ["DEEPSEEK_BASE_URL"])

    def extract_entity_from_query(query):
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "user", "content": f"What is the main entity or concept in this query: '{query}'? Extract only the core term (e.g., 'hyponatremia', not 'causes of hyponatremia'). Return only the entity name in one line."}]
        )
        entity = response.choices[0].message.content.strip()
        print(f'Extracted entity: {entity}')
        return entity
    
    return extract_entity_from_query

def llmself_tool():
    knowledge_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledgeable AI. Answer concisely and precisely using only your internal knowledge. No external context."),
        ("human", "{input}")
    ])
    knowledge_chain = LLMChain(llm=llm, prompt=knowledge_prompt)

    def knowledge_agent(question: str):
        """Agent that answers using only its internal knowledge"""
        return knowledge_chain.invoke({"input": question})['text']
    return knowledge_agent

## Worker nodes (Query Agent)
def planner_node(state: QState):
    question = next(
        (msg.content for msg in reversed(state["messages"]) 
         if msg.type == "human" and not hasattr(msg, 'name')), 
        None
    )
    
    if question is None:
        question = next(
            (msg.content for msg in state["messages"] if msg.type == "human"),
            "No question found"
        )
    
    planner_agent_executor = planner_tool()
    result = planner_agent_executor(question)
    return {
        "messages": [
            HumanMessage(content=result, name="planner")
        ],
        "planner_executed": True
    }

def multi_step_node(state: QState):
    original_question = None
    for msg in state["messages"]:
        if msg.type == "human" and not hasattr(msg, 'name'):
            original_question = msg.content
            break
    
    if not original_question:
        for msg in state["messages"]:
            if msg.type == "human":
                original_question = msg.content
                break
    
    if not original_question:
        original_question = "No question found"
    
    subquery_prompt = f"""
    Original query: {original_question}

    \nBreak the medical query into exactly two sub-queries. 
    Each sub-query should: 
    Be SIMPLE and SPECIFIC; 
    Focus on a different, narrow aspect of the original query;
    Avoid long or reasoning-based formulations. 
    
    \nFormat your response as: 
    1. [sub-query 1] 
    2. [sub-query 2]
    """
    
    result = llm.invoke(subquery_prompt)
    
    return {
        "messages": [
            HumanMessage(content=result.content, name="subquery")
        ]
    }

def query_agent_supervisor_node(state: QState):
    return {"messages": state["messages"]}

## Worker nodes (Knowledge Agent) - Modified for reference retrieval only
def lmkg_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    lmkg_agent_executor, lmkg_retriever = lmkg_retrieval_tool()
    response = lmkg_agent_executor.invoke({"input": question})
    result = response['output']

    # Get intermediate_steps from response
    intermediate_steps = response.get("intermediate_steps", [])
    
    def extract_reference_from_steps(intermediate_steps):
        """Extract reference information from agent steps, prioritizing non-empty results"""
        vector_reference = None
        cypher_reference = None
        cypher_has_results = False
        
        for step in intermediate_steps:
            if isinstance(step, tuple) and len(step) >= 2:
                tool_name = step[0].tool if hasattr(step[0], 'tool') else str(step[0])
                tool_result = step[1]
                
                if isinstance(tool_result, dict):
                    # Handle vector_qa tool
                    if 'med_vector_qa' in tool_name or 'vector' in tool_name.lower():
                        retrieved_context = tool_result.get("retrieved_context", [])
                        if retrieved_context and any(ctx.strip() for ctx in retrieved_context):
                            vector_reference = retrieved_context
                    
                    # Handle cypher_qa tool
                    if 'med_cypher_qa' in tool_name or 'cypher' in tool_name.lower():
                        generated_cypher = tool_result.get("generated_cypher", "")
                        retrieved_result = tool_result.get("retrieved_result", "")
                        
                        # Check if cypher has meaningful results
                        if retrieved_result and retrieved_result != [] and any(str(item).strip() for item in (retrieved_result if isinstance(retrieved_result, list) else [retrieved_result])):
                            cypher_has_results = True
                        
                        # Store cypher information regardless of whether result is empty
                        cypher_reference = {
                            "generated_cypher": generated_cypher or "N/A",
                            "retrieved_result": retrieved_result
                        }

        
        # Priority logic:
        # 1. If both tools used and cypher has results → return cypher reference
        # 2. If both tools used but cypher has no results → return vector reference
        # 3. If only cypher used → return cypher reference (even if empty)
        # 4. If only vector used → return vector reference
        # 5. If neither used → return "No information retrieved"
        
        if cypher_reference and vector_reference:  # Both tools were used
            if cypher_has_results:
                return cypher_reference  # Cypher has results, use it
            else:
                return vector_reference  # Cypher empty, use vector
        elif cypher_reference:  # Only cypher was used
            return cypher_reference
        elif vector_reference:  # Only vector was used
            return vector_reference
        else:  # No tools were used
            return "No information retrieved"
    
    reference = extract_reference_from_steps(intermediate_steps)

    # Update references and mark as completed    
    references = state.get("references", {})
    references["lmkg"] = reference

    completed_agents = state.get("completed_agents", [])
    if "lmkg" not in completed_agents:
        completed_agents.append("lmkg")
    
    return {
        "messages": [
            HumanMessage(content=f"LMKG reference retrieved", name="lmkg")
        ],
        "completed_agents": completed_agents,
        "references": references  # Fixed: was "reference" instead of "references"
    }

def hkg_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Only retrieve reference, no LLM processing
    hkg_retriever = hkg_retrieval_tool()
    response = hkg_retriever.similarity_search(question, k=1)
    reference = response[0].page_content if response else "No reference found"
    
    references = state.get("references", {})
    references["hkg"] = reference

    completed_agents = state.get("completed_agents", [])
    if "hkg" not in completed_agents:
        completed_agents.append("hkg")
    
    return {
        "messages": [
            HumanMessage(content=f"HKG reference retrieved", name="hkg")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def ds_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Only retrieve reference, no LLM processing
    ds_retriever = ds_retrieval_tool()
    response = ds_retriever.similarity_search(question, k=1)
    reference = response[0].page_content if response else "No reference found"
    
    references = state.get("references", {})
    references["ds"] = reference

    completed_agents = state.get("completed_agents", [])
    if "ds" not in completed_agents:
        completed_agents.append("ds")
    
    return {
        "messages": [
            HumanMessage(content=f"DS reference retrieved", name="ds")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def primekg_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Only retrieve reference, no LLM processing
    primekg_retriever = primekg_retrieval_tool()
    response = primekg_retriever.similarity_search(question, k=1)
    reference = response[0].page_content if response else "No reference found"
    
    references = state.get("references", {})
    references["primekg"] = reference

    completed_agents = state.get("completed_agents", [])
    if "primekg" not in completed_agents:
        completed_agents.append("primekg")
    
    return {
        "messages": [
            HumanMessage(content=f"PrimeKG reference retrieved", name="primekg")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def drugreviews_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Only retrieve reference, no LLM processing
    drugreviews_retriever = drugreviews_retrieval_tool()
    response = drugreviews_retriever.similarity_search(question, k=1)
    reference = response[0].page_content if response else "No reference found"
    
    references = state.get("references", {})
    references["drugreviews"] = reference

    completed_agents = state.get("completed_agents", [])
    if "drugreviews" not in completed_agents:
        completed_agents.append("drugreviews")
    
    return {
        "messages": [
            HumanMessage(content=f"drugreviews reference retrieved", name="drugreviews")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def wiki_node(state: State):
    from wiki_crawler import crawl_wikipedia_entity
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Wiki data retrieval
    query_entity = wiki_retrieval_tool()
    entity = query_entity(question)
    wiki_text = crawl_wikipedia_entity(entity, articles_limit=1)

    # Store as reference
    reference = wiki_text[:1500]
    references = state.get("references", {})
    references["wiki"] = reference

    completed_agents = state.get("completed_agents", [])
    if "wiki" not in completed_agents:
        completed_agents.append("wiki")
    
    return {
        "messages": [
            HumanMessage(content=f"Wiki reference retrieved", name="wiki")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def mayoclinic_node(state: State):
    from mayoclinic_crawler import crawl_mayoclinic_entity
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # MayoClinic data retrieval
    query_entity = mayoclinic_retrieval_tool()
    entity = query_entity(question)
    mayoclinic_text = crawl_mayoclinic_entity(entity)

    # Store as reference
    reference = mayoclinic_text
    references = state.get("references", {})
    references["mayoclinic"] = reference

    completed_agents = state.get("completed_agents", [])
    if "mayoclinic" not in completed_agents:
        completed_agents.append("mayoclinic")
    
    return {
        "messages": [
            HumanMessage(content=f"Mayo Clinic reference retrieved", name="mayoclinic")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

def llmself_node(state: State):
    question = state.get("original_question") or next(
        msg.content for msg in state["messages"] 
        if msg.type == "human"
    )
    
    # Generate LLM answer using internal knowledge
    llmself_agent_executor = llmself_tool()
    result = llmself_agent_executor(question)
    
    # Store the LLM-generated answer directly as the reference
    references = state.get("references", {})
    references["llmself"] = result  # Store the actual answer as reference

    completed_agents = state.get("completed_agents", [])
    if "llmself" not in completed_agents:
        completed_agents.append("llmself")
    
    return {
        "messages": [
            HumanMessage(content=f"LLM Self Agent completed", name="llmself")
        ],
        "completed_agents": completed_agents,
        "references": references
    }

# Create supervisor and verifier nodes
knowledge_agent_supervisor_node = make_supervisor_node(
    llm, 
    ["lmkg", "hkg", "ds", "primekg", "drugreviews", "wiki", "mayoclinic", "llmself"],
    agent_scopes={
        "lmkg": "General medical queries using knowledge graph: diseases, exams, indicators, symptoms, complications etc.",
        "hkg": "For questions on symptom.",
        "ds": "For questions on symptoms or treatments.",
        "primekg": "To retrieve drugs indicated for diseases.",
        "drugreviews": "To retrieve drugs along with patient reviews.",
        "wiki": "For disease definitions and overviews.",
        "mayoclinic": "For clinical info: causes, treatments, symptoms, complications.",
        "llmself": "Fallback only if others are irrelevant."
    }
)

# Create graph (Query Agent) - Modified to bypass supervisor
# Modified planner_node to handle single-step directly
def planner_node(state: QState):
    question = next(
        (msg.content for msg in reversed(state["messages"]) 
         if msg.type == "human" and not hasattr(msg, 'name')), 
        None
    )
    
    if question is None:
        question = next(
            (msg.content for msg in state["messages"] if msg.type == "human"),
            "No question found"
        )
    
    planner_agent_executor = planner_tool()
    result = planner_agent_executor(question)
    
    # Check if it's single-step and preserve the original question
    if "single-step" in result.lower():
        return {
            "messages": [
                HumanMessage(content=result, name="planner"),
                HumanMessage(content=question, name="single_step")  # Preserve original question
            ],
            "planner_executed": True
        }
    else:
        return {
            "messages": [
                HumanMessage(content=result, name="planner")
            ],
            "planner_executed": True
        }

query_builder = StateGraph(QState)
query_builder.add_node("planner", planner_node)
query_builder.add_node("multi_step", multi_step_node)

# Start directly with planner
query_builder.add_edge(START, "planner")

# Conditional edges from planner to multi_step or single_step
query_builder.add_conditional_edges(
    "planner",
    lambda state: determine_planner_decision(state),
    {
        "single_step": END,
        "multi_step": "multi_step"
    }
)

query_builder.add_edge("multi_step", END)

def determine_planner_decision(state: QState):
    planner_message = None
    for msg in reversed(state["messages"]):
        if hasattr(msg, 'name') and msg.name == "planner":
            planner_message = msg
            break
    
    if planner_message:
        planner_decision = planner_message.content.strip().lower()
        
        if "single-step" in planner_decision:
            return "single_step"
        elif "multi-step" in planner_decision:
            return "multi_step"
    
    # Default fallback
    return "single_step"

query_agent_graph = query_builder.compile()

## Create graph (Knowledge Agent)
knowledge_builder = StateGraph(State)
knowledge_builder.add_node("knowledge_supervisor", knowledge_agent_supervisor_node)
knowledge_builder.add_node("lmkg", lmkg_node)
knowledge_builder.add_node("hkg", hkg_node)
knowledge_builder.add_node("ds", ds_node)
knowledge_builder.add_node("primekg", primekg_node)
knowledge_builder.add_node("drugreviews", drugreviews_node)
knowledge_builder.add_node("wiki", wiki_node)
knowledge_builder.add_node("mayoclinic", mayoclinic_node)
knowledge_builder.add_node("llmself", llmself_node)

# Start with supervisor
knowledge_builder.add_edge(START, "knowledge_supervisor")

# Conditional edges FROM supervisor to workers and end
knowledge_builder.add_conditional_edges(
    "knowledge_supervisor",
    lambda state: state["next"],
    {
        "lmkg": "lmkg",
        "hkg": "hkg",
        "primekg": "primekg", 
        "wiki": "wiki",
        "mayoclinic": "mayoclinic",
        "llmself": "llmself",
        "ds": "ds",
        "drugreviews": "drugreviews",
        "FINISH": END,
        END: END
    }
)

# Unconditional edges FROM workers back TO supervisor
knowledge_builder.add_edge("lmkg", "knowledge_supervisor")
knowledge_builder.add_edge("hkg", "knowledge_supervisor")
knowledge_builder.add_edge("ds", "knowledge_supervisor")
knowledge_builder.add_edge("primekg", "knowledge_supervisor")
knowledge_builder.add_edge("drugreviews", "knowledge_supervisor")
knowledge_builder.add_edge("wiki", "knowledge_supervisor")
knowledge_builder.add_edge("mayoclinic", "knowledge_supervisor")
knowledge_builder.add_edge("llmself", "knowledge_supervisor")

knowledge_agent_graph = knowledge_builder.compile()

In [3]:
## RAG Chain
## Helper function (linker)
def process_final_message(final_message):
    stripped_message = final_message.strip()
    
    if '\n' in stripped_message:
        return [line.strip() for line in stripped_message.split('\n') if line.strip()]
    else:
        return [stripped_message]

## RAG Function
def rag(test_query):

    print("="*50)
    print(f"[Test query]: {test_query}")

    # Query Agent
    message_contents = []
    for s in query_agent_graph.stream(
        {"messages": [("user", test_query)]},
        {"recursion_limit": 100},
    ):
        
        if 'messages' in str(s):
            for key, value in s.items():
                if isinstance(value, dict) and 'messages' in value:
                    for msg in value['messages']:
                        content = msg.content
                        message_contents.append(content)

    final_message = message_contents[-1]
    message_list = process_final_message(final_message)

    # Knowledge Agent
    retrieved_references = {}

    for msg in message_list:
        final_state = None
        
        for s in knowledge_agent_graph.stream(
            {"messages": [("user", msg)]},
            {"recursion_limit": 100}):
            
            for key, value in s.items():
                if isinstance(value, dict):
                    if 'references' in value:
                        if final_state is None:
                            final_state = {}
                        final_state.update(value)

        # Print final results
        if final_state:
            if 'references' in final_state:
                retrieved_references[msg] = final_state['references']

    print(f"\n[Retrieved reference]: {retrieved_references}")
    return retrieved_references

In [ ]:
## Context
import json

df = pd.read_csv("../new_data/medqa_510_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "input"]
        if pd.notna(query):
            result = rag(query)
            result_str = json.dumps(result, ensure_ascii=False) # convert dict to string
            df.loc[i, "reference"] = result_str
            df.to_csv("../new_data/medqa_510_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

[Test query]: Q:A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?? 
{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Ciprofloxacin', 'D': 'Doxycycline', 'E': 'Nitrofurantoin'},
Question: 1. What is the most appropriate antibiotic treatment for a urinary tract infection in a pregnant woman?
Remaining agents: ['lmkg', 'hkg', 'ds', 'primekg', 'drugreviews', 'wiki', 'mayoclinic', 'llmself']
LLM selected: ds
Valid options: ['FINISH', 'lmkg', 'hkg', 'ds', 'primekg', 'dru

c:\Users\Sin Yee\anaconda3\envs\rag-env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: The question requires specific information regarding potential complications after surgery for a fractured femur in older patients. This suggests a factual query about complications. I'll use the med_cypher_qa tool to look up this information. 

Action:
```
{
  "action": "med_cypher_qa",
  "action_input": "Potential complications after surgery for a fractured femur in older patients"
}
```Retrieved node id: 122467


> Entering new GraphCypherQAChain chain...


C:\Users\Sin Yee\AppData\Local\Temp\ipykernel_5776\1398338777.py:319: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  cypher_response = cypher_chain({
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' is no longer supported)} {position: line: 3, column: 7, offset: 55} for query: 'cypher\nMATCH (n)-[:Complication]->(complication)\nWHERE id(n) = 122467\nRETURN complication\n'


Generated Cypher:
cypher
MATCH (n)-[:Complication]->(complication)
WHERE id(n) = 122467
RETURN complication


> Finished chain.
cypher_response:
{'query': 'Potential complications after surgery for a fractured femur in older patients', 'schema': 'Node properties are the following:\nDisease {名称, name_list, 类型}\nSymptom {发病部位, 重要提醒, 别称, 名称, 简介, 流行病学, 检查项目, 英文名称, 遗传性, 是否医保, name_list, 类型}\nMedicinal Substance {名称, name_list, 类型}\nBody Structure {名称, name_list, 类型}\nRadiographic Exam {名称, name_list, 类型}\nLaboratory Exam {名称, name_list, 类型}\nTherapy and Assessment {名称, name_list, 类型}\nProcedure {名称, name_list, 类型}\nPhysical Exam {名称, name_list, 类型}\nBiochemical Indicator {别称, 化验类型, 名称, 类别, name_list, 类型}\nMedical Department {名称, name_list, 类型}\nMedicinal Product {名称, name_list, 类型}\nOrganism {名称, name_list, 类型}\nPerson {from, name, kloutScore}\nDatabase {name}\nMessage {name}\nHOMO {name}\nLUMO {name}\nPiBond {name}\nInteraction {type}\nChargeTransportChannel {name}\nConductingNature {name}

...



> Entering new AgentExecutor chain...
```json
{
  "action": "med_vector_qa",
  "action_input": "What are the potential risk factors for skin cancer?"
}
```{'query': 'What are the potential risk factors for skin cancer?', 'retrieved_context': ['是否医保: 部分药物、耗材、诊治项目在医保报销范围，具体报销比例请咨询当地医院医保中心。@@dayi\n重要提醒: 皮肤癌早期经积极治疗后可预后良好，患者出现相关症状时应尽尽早就医处理。@@dayi,皮肤肿瘤的早期诊断和及时治疗，对患者的预后非常重要。@@dayi\n简介: 皮肤癌（cutaneous malignancies）是皮肤最常见的恶性肿瘤，是发生于身体暴露部位的恶性肿瘤的统称，可分为鳞状细胞癌、基底细胞癌、恶性黑色素瘤、湿疹样癌等多种类型，不同类型的皮肤癌症状特点可有一定差异，患者多有溃疡、丘疹、肿物等表现。本病多见于头面部及下肢。具有发展慢，恶性程度低，治愈率高等特点。@@dayi,皮肤肿瘤（skin tumors），是指发生于皮肤的各种良恶性肿瘤，为常见的皮肤病。本病的病因不明，可能与皮肤的组织解剖学因素、化学因素、物理因素、病毒因素、遗传因素等有关。患者的临床表现根据肿瘤类型不同而有所不同。皮肤肿瘤的治疗包括药物治疗、手术治疗、化学治疗、放射治疗和物理治疗。通常来讲，良性皮肤肿瘤，预后较好；而恶性皮肤肿瘤预后差异很大，视不同肿瘤而定。@@dayi\n名称: 皮肤癌\n检查项目: 体格检查@@dayi,病理检查@@dayi,皮肤镜检查@@dayi,血常规@@dayi,血生化@@dayi,超声@@dayi,CT@@dayi,MRI@@dayi,病理活检@@dayi\n别称: 皮肤肿瘤@@dayi\n相关中医疾病: 翻花疮@@dayi,恶疮@@dayi,石疔@@dayi,石疽@@dayi,癌疮@@dayi\n英文名称: cutaneous malignancies@@dayi,skin tumors@@dayi\nname_list: 皮肤癌,皮肤肿瘤,Neoplasm 

c:\Users\Sin Yee\anaconda3\envs\rag-env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: I need to determine which nerve is responsible for foot eversion and sensation on the lateral side of the leg and foot. This involves recalling the anatomical functions of specific nerves in the lower limb.

Action:
```
{
  "action": "med_cypher_qa",
  "action_input": "Which nerve is responsible for foot eversion and sensation on the lateral side of the leg and foot?"
}
```Retrieved node id: 99478


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)-[:Associated Symptom]->(nerve)
WHERE id(n) = 99478
RETURN nerve.name AS Nerve

{'error': '{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'Symptom\': expected\r\n  "*"\r\n  "WHERE"\r\n  "]"\r\n  "{"\r\n  a parameter (line 2, column 24 (offset: 30))\r\n"MATCH (n)-[:Associated Symptom]->(nerve)"\r\n                        ^}'}Thought: I encountered an error while attempting to retrieve data about the nerve responsible for foot eversion and sensa

c:\Users\Sin Yee\anaconda3\envs\rag-env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: I need to gather information about the key clinical features and laboratory findings associated with giardiasis. This query may require specific factual medical data regarding the disease.

Action:
```
{
  "action": "med_cypher_qa",
  "action_input": "What are the key clinical features and laboratory findings associated with giardiasis?"
}
```Retrieved node id: 346244


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)-[:Associated Symptom|:Associated Exam]->(m) 
WHERE id(n) = 346244 
RETURN m

{'error': '{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'Symptom\': expected\r\n  "*"\r\n  "WHERE"\r\n  "]"\r\n  "{"\r\n  a parameter (line 2, column 24 (offset: 30))\r\n"MATCH (n)-[:Associated Symptom|:Associated Exam]->(m)"\r\n                        ^}'}Thought: Since the specific query for clinical features and laboratory findings associated with giardiasis did not return successful results

In [ ]:
# Verifier & Summarizer (OpenAI)

In [ ]:
## Env setup
import getpass
import os
import sys
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv

sys.path.append(r'C:\Users\Sin Yee\Desktop\rag_techniques')

# LLM
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "You are a medical verifier AI.\n"
        "Your task is to:\n"
        "1. Assess the **relevance** and **quality** of each agent's reference\n"
        "2. Select suitable references that can be part of the context for the original question\n"
        "3. Remove irrelevant references (e.g. if mayoclinic's url has irrelevant keyword, or retrieved disease/drug is irrelevant, remove this reference)\n"
        "Output format:\n"
        "'selected_agents': list of agent names with the strongest references, do not modify the agent name\n\n"
        "DO NOT RESPONSE WITH ANYTHING ELSE.\n\n"
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference: \n{ref}")
])

verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as context to answer a single-choice question.\n\n"
        f"Answer with your own knowledge if the context is irrelevant or wrong.\n\n"
        f"ONLY response with your final choice in a single alphabet: A, B, C, D, or E.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

In [ ]:
## Result
df = pd.read_csv("../new_data/medqa_510_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "input"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references) # convert dict to string
            df.loc[i, "gpt_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "gpt-4o-mini"] = result["final_answer"]
            df.to_csv("../new_data/medqa_510_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

Example output:
==================================================
Processing question: 1. What embryologic error could cause the infant's symptoms of regurgitation and yellow vomit?
[Verifier response]:{'selected_agents': ['ds']}
Selected agents for this question: ['ds']
==================================================
Processing question: 2. What condition is indicated by the infant's fussy behavior and minimally distended abdomen?
[Verifier response]:{'selected_agents': ['hkg', 'ds']}
Selected agents for this question: ['hkg', 'ds']
[Verified references]:  {"1. What embryologic error could cause the infant's symptoms of regurgitation and yellow vomit?": {"ds": "Name: Neonatal Jaundice\nSymptoms: Yellowing of the skin and eyes, pale stools, dark urine\nTreatments: Phototherapy, exchange transfusion (in severe cases), monitoring of bilirubin levels, addressing underlying causes (e.g., breastfeeding)"}, "2. What condition is indicated by the infant's fussy behavior and minimally distended abdomen?": {"hkg": "Diseases: appendicitis\nSymptoms: pain (0.881), nausea (0.401), abdominal pain (0.361), tenderness (0.163), chills (0.152), diarrhea (0.124), vomiting (0.118), fever (0.096), loss of appetite (0.068), lower abdominal pain (0.040), cramping (0.037), constipation (0.036), discomfort (0.033), cyst (0.030), pain in right lower abdomen (0.029), sharp pain (0.023), pain during urination (0.022), pain in upper abdomen (0.020), pelvic pain (0.017), flank pain (0.016), vaginal discharge (0.013), abdominal discomfort (0.013), dull pain (0.012), infection (0.011)", "ds": "Name: Pyloric stenosis\nSymptoms: Persistent hunger, Weight loss, Dehydration, Visible peristalsis, Palpable mass, Fussiness\nTreatments: Ultrasound, Blood tests, Pyloromyotomy"}}
[Summarizer answer]:  D


In [ ]:
# Verifier & Summarizer (DeepSeek)

In [ ]:
## Env setup
import getpass
import os
import sys
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_deepseek import ChatDeepSeek
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv

sys.path.append(r'C:\Users\Sin Yee\Desktop\rag_techniques')

# LLM
load_dotenv()
os.environ["DEEPSEEK_API_KEY"] = os.getenv('DEEPSEEK_API_KEY')
os.environ["DEEPSEEK_BASE_URL"] = os.getenv('DEEPSEEK_BASE_URL')

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "You are a medical verifier AI.\n"
        "Your task is to:\n"
        "1. Assess the **relevance** and **quality** of each agent's reference\n"
        "2. Select suitable references that can be part of the context for the original question\n"
        "3. Remove irrelevant references (e.g. if mayoclinic's url has irrelevant keyword, remove this reference)\n"
        "Output format:\n"
        "'selected_agents': list of agent names with the strongest references, do not modify the agent name\n\n"
        "DO NOT RESPONSE WITH ANYTHING ELSE.\n\n"
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference: \n{ref}")
])

verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as context to answer a single-choice question.\n\n"
        f"ONLY answer with your own knowledge if the context is irrelevant to the query.\n\n"
        f"ONLY response with your final choice in a single alphabet: A, B, C, D, or E.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

In [ ]:
## Result
df = pd.read_csv("../new_data/medqa_510_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "input"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references) # convert dict to string
            df.loc[i, "ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "ds"] = result["final_answer"]
            df.to_csv("../new_data/medqa_510_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

Example output:
{'1. What are the psychological effects of surgical removal of a testicle on a male patient?': {'mayoclinic': {'url': 'https://www.mayoclinic.org/diseases-conditions/scrotal-masses/diagnosis-treatment/drc-20352609', 'Treatment': "Treatment of a scrotal mass mainly depends on its cause. Infections Medicines called antibiotics can treat a scrotal mass caused by bacteria, as is often the case with epididymitis. If a virus causes epididymitis or orchitis, the usual treatment involves rest, ice and pain relief medicine. Scrotal masses that aren't cancer You might hear these called benign masses. Sometimes they don't need treatment. Other times, they need to be removed with surgery, repaired or drained. Treatment decisions depend on factors such as whether the scrotal mass: Causes discomfort or pain.Contributes to or raises the risk of infertility.Becomes infected. Testicular cancer If your scrotal mass is caused by cancer that starts in a testicle, you'll likely see a cancer doctor called an oncologist. The oncologist may recommend treatments based on whether the cancer is in the testicle or has spread to other body parts. Your age and overall health also are factors. The main treatment choices include: Radical inguinal orchiectomy.This is the main treatment for testicular cancer. It's surgery to remove the affected testicle and the spermatic cord through a cut..."}, 'llmself': 'The surgical removal of a testicle, known as orchiectomy, can have several psychological effects on a male patient, including:\n\n1. **Body Image Issues**: Patients may experience changes in self-esteem and body image, feeling less masculine or self-conscious about their appearance.\n\n2. **Anxiety and Depression**: The surgery and its implications can lead to feelings of anxiety, sadness, or depression, particularly regarding fertility concerns or changes in sexual function.\n\n3. **Sexual Dysfunction**: Some men may experience anxiety related to sexual performance or changes in libido, which can impact intimate relationships.\n\n4. **Fear of Cancer Recurrence**: If the surgery is due to cancer, patients may have ongoing fears about cancer recurrence and additional health concerns.\n\n5. **Adjustment and Coping**: Individual coping mechanisms play a significant role; some may adapt well, while others may struggle with the emotional impact.\n\n6. **Support Needs**: Many benefit from psychological support or counseling to help process their feelings and gain coping strategies. \n\nOverall, the psychological effects can vary widely based on individual circumstances and support systems.'}, '2. What are examples of humor as a coping mechanism after a medical procedure?': {'llmself': 'Examples of humor as a coping mechanism after a medical procedure include:\n\n1. **Lighthearted Jokes**: Patients might joke about their condition or treatment, such as calling their hospital gown a "fashion statement."\n   \n2. **Comic Relief**: Sharing funny stories or anecdotes from hospital experiences with friends or family to lighten the mood.\n\n3. **Humorous Observations**: Making witty comments about the hospital environment or staff, like teasing about the food or sharing playful banter with nurses.\n\n4. **Memes and Cartoons**: Using funny images or memes related to medical experiences on social media to connect with others and share a laugh.\n\n5. **Funny Movies or Shows**: Watching comedies while recovering to distract from pain and create a relaxed atmosphere.\n\n6. **Playful Self-Deprecation**: Poking fun at oneself in relation to the recovery process, such as joking about needing a "personal assistant" for daily tasks. \n\nThese forms of humor can help reduce anxiety, foster social connections, and promote a positive outlook during recovery.'}}
==================================================
Processing question: 1. What are the psychological effects of surgical removal of a testicle on a male patient?
[Verifier response]:{'selected_agents': ['llmself']}
Selected agents for this question: ['llmself']
==================================================
Processing question: 2. What are examples of humor as a coping mechanism after a medical procedure?
[Verifier response]:{'selected_agents': ['llmself']}
Selected agents for this question: ['llmself']
[Verified references]:  {"1. What are the psychological effects of surgical removal of a testicle on a male patient?": {"llmself": "The surgical removal of a testicle, known as orchiectomy, can have several psychological effects on a male patient, including:\n\n1. **Body Image Issues**: Patients may experience changes in self-esteem and body image, feeling less masculine or self-conscious about their appearance.\n\n2. **Anxiety and Depression**: The surgery and its implications can lead to feelings of anxiety, sadness, or depression, particularly regarding fertility concerns or changes in sexual function.\n\n3. **Sexual Dysfunction**: Some men may experience anxiety related to sexual performance or changes in libido, which can impact intimate relationships.\n\n4. **Fear of Cancer Recurrence**: If the surgery is due to cancer, patients may have ongoing fears about cancer recurrence and additional health concerns.\n\n5. **Adjustment and Coping**: Individual coping mechanisms play a significant role; some may adapt well, while others may struggle with the emotional impact.\n\n6. **Support Needs**: Many benefit from psychological support or counseling to help process their feelings and gain coping strategies. \n\nOverall, the psychological effects can vary widely based on individual circumstances and support systems."}, "2. What are examples of humor as a coping mechanism after a medical procedure?": {"llmself": "Examples of humor as a coping mechanism after a medical procedure include:\n\n1. **Lighthearted Jokes**: Patients might joke about their condition or treatment, such as calling their hospital gown a \"fashion statement.\"\n   \n2. **Comic Relief**: Sharing funny stories or anecdotes from hospital experiences with friends or family to lighten the mood.\n\n3. **Humorous Observations**: Making witty comments about the hospital environment or staff, like teasing about the food or sharing playful banter with nurses.\n\n4. **Memes and Cartoons**: Using funny images or memes related to medical experiences on social media to connect with others and share a laugh.\n\n5. **Funny Movies or Shows**: Watching comedies while recovering to distract from pain and create a relaxed atmosphere.\n\n6. **Playful Self-Deprecation**: Poking fun at oneself in relation to the recovery process, such as joking about needing a \"personal assistant\" for daily tasks. \n\nThese forms of humor can help reduce anxiety, foster social connections, and promote a positive outlook during recovery."}}
C:\Users\Sin Yee\AppData\Local\Temp\ipykernel_27216\3322719192.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{"1. What are the psychological effects of surgical removal of a testicle on a male patient?": {"llmself": "The surgical removal of a testicle, known as orchiectomy, can have several psychological effects on a male patient, including:\n\n1. **Body Image Issues**: Patients may experience changes in self-esteem and body image, feeling less masculine or self-conscious about their appearance.\n\n2. **Anxiety and Depression**: The surgery and its implications can lead to feelings of anxiety, sadness, or depression, particularly regarding fertility concerns or changes in sexual function.\n\n3. **Sexual Dysfunction**: Some men may experience anxiety related to sexual performance or changes in libido, which can impact intimate relationships.\n\n4. **Fear of Cancer Recurrence**: If the surgery is due to cancer, patients may have ongoing fears about cancer recurrence and additional health concerns.\n\n5. **Adjustment and Coping**: Individual coping mechanisms play a significant role; some may adapt well, while others may struggle with the emotional impact.\n\n6. **Support Needs**: Many benefit from psychological support or counseling to help process their feelings and gain coping strategies. \n\nOverall, the psychological effects can vary widely based on individual circumstances and support systems."}, "2. What are examples of humor as a coping mechanism after a medical procedure?": {"llmself": "Examples of humor as a coping mechanism after a medical procedure include:\n\n1. **Lighthearted Jokes**: Patients might joke about their condition or treatment, such as calling their hospital gown a \"fashion statement.\"\n   \n2. **Comic Relief**: Sharing funny stories or anecdotes from hospital experiences with friends or family to lighten the mood.\n\n3. **Humorous Observations**: Making witty comments about the hospital environment or staff, like teasing about the food or sharing playful banter with nurses.\n\n4. **Memes and Cartoons**: Using funny images or memes related to medical experiences on social media to connect with others and share a laugh.\n\n5. **Funny Movies or Shows**: Watching comedies while recovering to distract from pain and create a relaxed atmosphere.\n\n6. **Playful Self-Deprecation**: Poking fun at oneself in relation to the recovery process, such as joking about needing a \"personal assistant\" for daily tasks. \n\nThese forms of humor can help reduce anxiety, foster social connections, and promote a positive outlook during recovery."}}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "ver_ref"] = verified_references_str
[Summarizer answer]:  E

In [ ]:
# MedMCQA's format query function

In [ ]:
import pandas as pd
df = pd.read_csv("../new_data/mcq/medmcqa_545_result.csv")

def format_query(ques):
    question = ques['question']
    options = [f"A. {ques['opa']}",
            f"B. {ques['opb']}",
            f"C. {ques['opc']}",
            f"D. {ques['opd']}"]
    query = f"{question} | {' | '.join(options)}"
    return query

ques = df.loc[0]
query = format_query(ques)
print(query)

In [ ]:
## Context
import json
df = pd.read_csv("../new_data/medmcqa_545_result.csv")
for i in range(len(df)):
    try:
        query = format_query(df.loc[i])
        if pd.notna(query):
            result = rag(query)
            result_str = json.dumps(result, ensure_ascii=False)
            df.loc[i, "reference"] = result_str
            df.to_csv("../new_data/medmcqa_545_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
## Verifier-Summarizer (gpt)
import json
import pandas as pd

df = pd.read_csv("../new_data/medmcqa_545_result.csv")
for i in range(len(df)):
    try:
        query = format_query(df.loc[i])
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                verifier_results = verifier_agent(question, agent_references)
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references, ensure_ascii=False)
            df.loc[i, "gpt_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "gpt"] = result["final_answer"]
            df.to_csv("../new_data/medmcqa_545_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue